In [0]:
%run "../../constants/configurations"

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
drivers_df = spark.read.parquet(f"{silver_container_path}")\
    .withColumnRenamed("number", "driver_number")\
    .withColumnRenamed("name", "driver_name")\
    .withColumnRenamed("nationality", "driver_nationality")


In [0]:
constructor_df = spark.read.parquet(f"{silver_container_path}")\
    .withColumnRenamed("name", "team")

In [0]:
circuits_df = spark.read.parquet(f"{silver_container_path}")\
    .withColumnRenamed("location", "circuit_location")

In [0]:
races_df = spark.read.parquet(f"{silver_container_path}")\
    .withColumnRenamed("name", "race_name")
    .withColumnRenamed("race_timpespam", "race_date")

In [0]:
results_df = spark.read.parquet(f"{silver_container_path}")\
    .withColumnRenamed("time", "race_time")

##### join circuit dataframe to races dataframe

In [0]:
race_circuit_df = races_df.join(circuits_df, races_df.circuit_id == circuit_df.circuit_id, "inner")\
  .select(races.df.race_id, races.df.race_year, races.df.race_name, races.df.date, circuits.df.circuit_location)

##### join results to all dataframe

In [0]:
race_results_df = results_df.join(race_circuit_df, results_df.race_id == races_df.race_id)\
                            .join(drivers_df, results_df.driver_id == drivers_df.driver_id)\
                            .join(constructor_df, results_df.constructor_id == constructor_df.constructor_id)

In [0]:
race_results_df = race_results_df.select( "race_year", "race_name", "race_date", "circuit_location", "driver_number", 
                                  "driver_name", "driver_nationality", "team", "grid" ,"fastest_lap","race_time", "points", "position")\
                                .withColumn("created_date", current_timestamp())

In [0]:
race_results_df.filter(race_results_df.race_year == 2020 and race_names.race_name == "Abu Dhabi Grand Prix").orderBy(final_df.points.desc())

In [0]:
##race_results_df.write.mode("overwrite").parquet(f"{gold_container_path}")